<a href="https://colab.research.google.com/github/ningxia202109/llm-learn/blob/main/python-langchain-tutorials/ollama-on-t4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10975    0 10975    0     0  32002      0 --:--:-- --:--:-- --:--:-- 32090
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InReleas

In [2]:
import os

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})

In [3]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2024-07-09 02:36:56--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.6.1/cloudflared-linux-amd64.deb [following]
--2024-07-09 02:36:56--  https://github.com/cloudflare/cloudflared/releases/download/2024.6.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/3e345268-c5d6-4324-8389-71790dcf95ac?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20240709%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20240709T023656Z&X-Amz-Expires=300&X-Amz-Signature=c3feb00d9dec00dd9274fef44cd7e7fb48d8432fe6009036493e14895cbb

In [4]:
import subprocess
import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("\n\n\n\n\n")
            print("running ollama server\n\n", l[l.find("http"):], end='')
            print("\n\n\n\n\n")

threading.Thread(target=iframe_thread, daemon=True, args=(11434,)).start()

In [6]:
!ollama serve > ollama-server.log 2>&1 &
!ollama run qwen2 > qwen2.log 2>&1 &
!sleep 120
!ollama list

NAME        	ID          	SIZE  	MODIFIED       
qwen2:latest	e0d4e1163c58	4.4 GB	45 seconds ago	


In [8]:
!pip install -U -q langchain
!pip install -U -q langchain_community

In [10]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize an instance of the Ollama model
llm = Ollama(model="qwen2")

# Using Language Models
messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="Today is a good day"),
]

# Invoke the model to generate responses
response = llm.invoke(messages)
print(response)

今天是个好日子。
